In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression

In [26]:
data = pd.DataFrame(columns= ['Name', 'Year','Seed','Champion','MVP','Scoring Leader', 
                              'Rebound Leader', 'Assist Leader', 'WS Leader','DPOY',
                              'MIP', '6MOY', #'All-Stars', 'All-NBA', 'FG%', '3P%', 
                              #'2P%', 'FT%', 'PPG', 'TOV', 'AST', 'STL', 'BLK' 
                              #'SEED', 'WINS', 'GOAT Factor', 'Coach', 'Overall Team Score' 
                             ])
data

,Name,Year,Seed,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,MIP,6MOY


In [22]:
def check_roster(team_soup, season_awards):
    awards = [0,0,0,0,0,0,0,0]
    players = team_soup.find_all('td', {'class':'left', 'data-stat':'player'})
    roster = [player.text for player in players]
    
    for i in range(5):
        if season_awards[i] in roster:
            awards[i] = 1
    
    return awards

In [23]:
def get_team_score(season_awards, name, team_url, year, seed):
    row = []
    row.append(name.replace('*',''))
    row.append(year)
    row.append(seed)
    
    roster = []
    
    url = 'http://www.basketball-reference.com' + team_url
    req = requests.get(url)
    time.sleep(0.1)
    team_soup = BeautifulSoup(req.text, 'html.parser')
    try:
        row.append(team_soup.find('br').text[:4] == 'Won ')
    except:
        row.append(False)
    row += check_roster(team_soup, season_awards)
    return row

In [24]:
def get_award_winners(season):
    MVP = season.body.find(text='Most Valuable Player').parent.parent.find('a').text
    score_champ = season.body.find(text='PPG Leader').parent.parent.find('a').text
    rbd_champ = season.body.find(text='RPG Leader').parent.parent.find('a').text
    assist_champ = season.body.find(text='APG Leader').parent.parent.find('a').text
    ws_leader = season.body.find(text='WS Leader').parent.parent.find('a').text


    return [MVP,score_champ, rbd_champ, assist_champ, ws_leader]

In [30]:
row_num = 0
for i in range(1996, 1997):
    url = ('http://www.basketball-reference.com/leagues/NBA_%s.html' % str(i))
    time.sleep(0.1)
    req = requests.get(url)
    season = BeautifulSoup(req.text, 'html.parser')
    teams = season.find_all('th', {'class': 'left', 'data-stat':'team_name'})
   
    season_awards = get_award_winners(season)
    print(season_awards)
    for team in teams:
        try:
            seed = int(team.text[-3])
        except:
            pass
        team_name = repr(team.text).replace('\\x','*').split('*')[0].lstrip("'")
        print(team_name)
        for a in team.find_all('a',href=True):
            new_row = get_team_score(season_awards,team_name,a['href'], i, seed)
            print(new_row)
            data.loc[row_num] = new_row
            row_num += 1


['Michael Jordan', 'Michael Jordan', 'Dennis Rodman', 'John Stockton', 'Michael Jordan']
Eastern Conference'
Orlando Magic
['Orlando Magic', 1996, 2, False, 0, 0, 0, 0, 0, 0, 0, 0]
New York Knicks
['New York Knicks', 1996, 5, False, 0, 0, 0, 0, 0, 0, 0, 0]
Miami Heat
['Miami Heat', 1996, 8, False, 0, 0, 0, 0, 0, 0, 0, 0]
Washington Bullets
['Washington Bullets', 1996, 0, False, 0, 0, 0, 0, 0, 0, 0, 0]
Boston Celtics
['Boston Celtics', 1996, 1, False, 0, 0, 0, 0, 0, 0, 0, 0]
New Jersey Nets
['New Jersey Nets', 1996, 2, False, 0, 0, 0, 0, 0, 0, 0, 0]
Philadelphia 76ers
['Philadelphia 76ers', 1996, 5, False, 0, 0, 0, 0, 0, 0, 0, 0]
Chicago Bulls
['Chicago Bulls', 1996, 1, True, 1, 1, 1, 0, 1, 0, 0, 0]
Indiana Pacers
['Indiana Pacers', 1996, 3, False, 0, 0, 0, 0, 0, 0, 0, 0]
Cleveland Cavaliers
['Cleveland Cavaliers', 1996, 4, False, 0, 0, 0, 0, 0, 0, 0, 0]
Atlanta Hawks
['Atlanta Hawks', 1996, 6, False, 0, 0, 0, 0, 0, 0, 0, 0]
Detroit Pistons
['Detroit Pistons', 1996, 7, False, 0, 0, 0, 0

In [31]:
data

,Name,Year,Seed,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,MIP,6MOY
-1,Los Angeles Clippers,1996.0,1.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Orlando Magic,1996.0,2.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,New York Knicks,1996.0,5.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Miami Heat,1996.0,8.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Washington Bullets,1996.0,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Boston Celtics,1996.0,1.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,New Jersey Nets,1996.0,2.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Philadelphia 76ers,1996.0,5.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Chicago Bulls,1996.0,1.0,True,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
8,Indiana Pacers,1996.0,3.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
